<a href="https://colab.research.google.com/github/superspray/11/blob/master/MRC_extractor%26make_tables_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import warnings
warnings.filterwarnings("ignore")
!pip install transformers==3.3.0
from transformers import AutoTokenizer, AutoModelForSequenceClassification
!pip install word2number
#import mrc_extractor




In [ ]:
#queries 로 변경 

import numpy as np
import warnings
warnings.filterwarnings("ignore")
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline


class MRC_EXTRACTION:

	def __init__(self, queries, full_context, max_seq_len=512, \
              trained_model1="amberoad/bert-multilingual-passage-reranking-msmarco", \
              trained_model2="ahotrod/electra_large_discriminator_squad2_512"):
		self.queries = queries
		#self.query = query
		self.full_context = full_context
		self.max_seq_len =  max_seq_len
		self.trained_model1 = trained_model1
		self.trained_model2 = trained_model2
		self.tokenizer = AutoTokenizer.from_pretrained(self.trained_model1)
		self.model = AutoModelForSequenceClassification.from_pretrained(self.trained_model1)
		self.model_name = self.trained_model2
				#deepset/electra-base-squad2 
				#twmkn9/distilbert-base-uncased-squad2
		self.nlp = pipeline('question-answering', model=self.model_name, tokenizer=self.model_name)


	def get_sep_passages(self):
		# <br> 처리 & ' 처리된 text
		context=[ x for x in self.full_context.split('<br>') if x]
		n_text=len(context)
		n_token_query = max([len(self.tokenizer(query)['input_ids']) for query in self.queries]) + 1

		i=0
		texts=[]
		while i<= n_text-1:
			index=i+1
			while index<=n_text-1:
				if len(self.tokenizer(" ".join(context[i:index+1]))['input_ids']) > self.max_seq_len-n_token_query:
					break;
				index += 1
			texts.append(" ".join(context[i: index]))
			i=index
		return texts

	# Model 1 
	def passage_reranking(self):

		texts = self.get_sep_passages()
		ranked_texts = {}

		for query in self.queries:
			scores = []
			for text in texts: 
				inputs = self.tokenizer.encode_plus(query, text, truncation=True, add_special_tokens=True, return_tensors="pt",max_length=512)
				input_ids = inputs["input_ids"].tolist()[0]
				score = self.model(**inputs)[0][0][1]
				scores.append(score)
			ranked_texts[query] = np.array(texts)[np.array(scores).argsort()[::-1]]

		return ranked_texts

	# arg 형태 및 에러 관련 수정사항 추가할 것
	# Model 2 
	def get_answer(self, query, context):

		QA_input = {
		'question': query,
		'context': context
		}
		res = self.nlp(QA_input, model_max_length=self.max_seq_len) # max_question_len=64
		return query, res['answer'], res['score']

	def get_topN_answers(self, max_texts=3, n_answer=1):

		texts = self.passage_reranking()
		n_texts=len(texts[list(texts.keys())[0]])

		qa_result={}
		for query in self.queries:
			res={}
			for i in range(min(n_texts, max_texts)):
				result = self.get_answer(query=query, context=texts[query][i])
				res[i] = (result[1], result[2])
			res = sorted(res.items(), key=lambda x: x[1][1], reverse=True)
			answers = [x[1][0] for x in res[:n_answer]] #candidates
			if n_answer==1:
				answers=answers[0]
			qa_result[query]=answers
		return qa_result





In [ ]:
full="2021.02.14. 16:03 We're starting 3rd KIC Project Progress Check meeting.  The attendees of today's meeting are Erin, Kim, Steven, Sierra, Koo, and Jeon.   Shall we begin our meeting?  So we have two main agendas for today.  Why don't we start with Kim and talk about the first agenda?    Kim  The first thing we need to discuss about is what API we should use.  So I have been searching up APIs and found 5 products, but the ones worth talking about are 2, one made by Google and the other made by IBM. Both of them have pretty much similar quality I think. They can interpret the words well and transcribe them with high accuracy. The bottom line is, Google and IBM works the best.     Erin  Oh I have used Google API before. Isn't it kinda bad? The accuracy is not so great isn't it?     Kim  I think the accuracy and the level of completion is different between the free version and premium paid version. Since we can still use the premium paid version for free during trial period, we should do that.    Erin   Alright then, could you please use the premium version and see how it works out? After that let's select the API and do some coding works to get them to function. You should work with Sierra on that Can we do this by next meeting which we should hold on the 12th?    Kim   Okay, so we have two action items for this. First thing is to evaluate Google Premium API, and i will do that. The next item is to confirm API and edit the code, me, Kim and Sierra will    Erin  Next is Steven, please update us with your interface works, the second agenda for this meeting.     Steven  Well I haven't been very productive with my task so please excuse me. What I did do however is I searched up some already-available services for Ai automated transcription services. There were two that I found to be somewhat relevant to what we were doing: Otter.ai and Deeptalk. Fortunately both of them have yet developed ways to produce meeting notes or minute so if we can get it done, we would be the first to do so. I will get the interface design by today and the rest of the work including homepage feedback by the next meeting on the 12th.    Erin   alright. Steven will have two tasks for the second agenda. First one is to get the interface design by today. Second is to give some Homepage feedback by the next meeting.  "

sj=MRC_EXTRACTION(query, full)

sj.get_sep_passages()

["2021.02.14. 16:03 We're starting 3rd KIC Project Progress Check meeting.  The attendees of today's meeting are Erin, Kim, Steven, Sierra, Koo, and Jeon.   Shall we begin our meeting?  So we have two main agendas for today.  Why don't we start with Kim and talk about the first agenda?    Kim  The first thing we need to discuss about is what API we should use.  So I have been searching up APIs and found 5 products, but the ones worth talking about are 2, one made by Google and the other made by IBM. Both of them have pretty much similar quality I think. They can interpret the words well and transcribe them with high accuracy. The bottom line is, Google and IBM works the best.     Erin  Oh I have used Google API before. Isn't it kinda bad? The accuracy is not so great isn't it?     Kim  I think the accuracy and the level of completion is different between the free version and premium paid version. Since we can still use the premium paid version for free during trial period, we should do

In [ ]:
#full="2021.02.14. 16:03  Erin  Hi everyone, We're starting 3rd KIC Project Progress Check meeting.  The attendees of today's meeting are Erin, Kim, Steven, Sierra, Koo, and Jeon.   Shall we begin our meeting?  So we have two main agendas for today.  Why don't we start with Kim and talk about the first agenda?    Kim  The first thing we need to discuss about is what API we should use.  So I have been searching up APIs and found 5 products, but the ones worth talking about are 2, one made by Google and the other made by IBM. Both of them have pretty much similar quality I think. They can interpret the words well and transcribe them with high accuracy. The bottom line is, Google and IBM works the best.     Erin  Oh I have used Google API before. Isn't it kinda bad? The accuracy is not so great isn't it?     Kim  I think the accuracy and the level of completion is different between the free version and premium paid version. Since we can still use the premium paid version for free during trial period, we should do that.    Erin   Alright then, could you please use the premium version and see how it works out? After that let's select the API and do some coding works to get them to function. You should work with Sierra on that Can we do this by next meeting which we should hold on the 12th?    Kim   Okay, so we have two action items for this. First thing is to evaluate Google Premium API, and i will do that. The next item is to confirm API and edit the code, me, Kim and Sierra will    Erin  Next is Steven, please update us with your interface works, the second agenda for this meeting.     Steven  Well I haven't been very productive with my task so please excuse me. What I did do however is I searched up some already-available services for Ai automated transcription services. There were two that I found to be somewhat relevant to what we were doing: Otter.ai and Deeptalk. Fortunately both of them have yet developed ways to produce meeting notes or minute so if we can get it done, we would be the first to do so. I will get the interface design by today and the rest of the work including homepage feedback by the next meeting on the 12th."
query=['who are the attendees at the meeting?',
	'what was the main topic of the meeting?'
	]

mrc=MRC_EXTRACTION(query, full)


In [ ]:
answers=mrc.get_topN_answers()
answers

{'what was the conclusion of the first agenda?': 'Google and IBM works the best.',
 'what was the first action item of the first agenda?': 'evaluate Google Premium API,',
 'what was the main topic of the meeting?': 'KIC Project Progress Check',
 'what was the topic of the first agenda?': 'what API we should use.',
 'when is the deadline of the first action item?': 'evaluate Google Premium API,',
 'who are the attendees at the meeting?': 'Erin, Kim, Steven, Sierra, Koo, and Jeon.',
 'who is responsible for the first action item?': 'Kim',
 'who was the presenter of the first agenda?': 'Kim'}

In [ ]:
import pandas as pd
questions={'table_top': {
	'Participants':'who are the attendees at the meeting?',
	'Topic':'what was the main topic of the meeting?'
	},
 'table_mid_v1': {
	'Discussion Topic':'what was the topic of the first agenda?',
	'Presenter':'who was the presenter of the first agenda?',
	'Conclusion':'what was the conclusion of the first agenda?'
	},
 'table_mid_v2': {
	'Action Items':'what was the first ?',
	'Person Responsible':'who was the presenter of the first agenda?',
	'Deadline':'what was the deadline of the first action item?'
	}
}
df=pd.DataFrame(di)
df

,table_top,table_mid_v1,table_mid_v2
Participants,who are the attendees at the meeting?,NaN,NaN
Topic,what was the main topic of the meeting?,NaN,NaN
Discussion Topic,NaN,what was the topic of the first agenda?,NaN
Presenter,NaN,who was the presenter of the first agenda?,NaN
Conclusion,NaN,what was the conclusion of the first agenda?,NaN
Action Items,NaN,NaN,what was the first ?
Person Responsible,NaN,NaN,who was the presenter of the first agenda?
Deadline,NaN,NaN,what was the deadline of the first action item?


In [ ]:
df=df.replace(answers)
df

,table_top,table_mid_v1,table_mid_v2
Participants,"Erin, Kim, Steven, Sierra, Koo, and Jeon.",NaN,NaN
Topic,KIC Project Progress Check,NaN,NaN
Discussion Topic,NaN,what API we should use.,NaN
Presenter,NaN,Kim,NaN
Conclusion,NaN,Google and IBM works the best.,NaN
Action Items,NaN,NaN,what was the first ?
Person Responsible,NaN,NaN,Kim
Deadline,NaN,NaN,what was the deadline of the first action item?


In [ ]:
df.to_dict()

{'table_mid_v1': {'Action Items': nan,
  'Conclusion': 'Google and IBM works the best.',
  'Deadline': nan,
  'Discussion Topic': 'what API we should use.',
  'Participants': nan,
  'Person Responsible': nan,
  'Presenter': 'Kim',
  'Topic': nan},
 'table_mid_v2': {'Action Items': 'what was the first ?',
  'Conclusion': nan,
  'Deadline': 'what was the deadline of the first action item?',
  'Discussion Topic': nan,
  'Participants': nan,
  'Person Responsible': 'Kim',
  'Presenter': nan,
  'Topic': nan},
 'table_top': {'Action Items': nan,
  'Conclusion': nan,
  'Deadline': nan,
  'Discussion Topic': nan,
  'Participants': 'Erin, Kim, Steven, Sierra, Koo, and Jeon.',
  'Person Responsible': nan,
  'Presenter': nan,
  'Topic': 'KIC Project Progress Check'}}

In [ ]:
import numpy as np
import warnings
warnings.filterwarnings("ignore")
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
import mrc_extractor
from mrc_extractor import MRC_EXTRACTION
import pandas as pd
from word2number import w2n

query_dict_ex={'table_top': {
	'Participants':'who are the attendees at the meeting?',
	'Topic':'what was the main topic of the meeting?',
    'num_agendas':'How many agendas?',
    'num_act_items_1':'How many action items for the first agenda?', # 이후 일괄 처리 --> 질문 생성 함수 만들기
    'num_act_items_2':'How many action items for the second agenda?' # 이후 일괄 처리 --> 질문 생성 함수 만들기
	},
 'table_main1': {
    'Discussion Topic':'what was the topic of the first agenda?',
	'Presenter':'who was the presenter of the first agenda?',
	'Conclusion':'what was the conclusion of the first agenda?'
 },
  'table_main1_1':{
	'Action Item':'what was the first task for the first agenda ?',
	'Person Responsible':'who was responsible for the first task  of the first agenda?',
	'Deadline':'when is the deadline of the first task of the first agenda?'
  },
    'table_main1_2':{
	'Action Item':'what was the second task for the first agenda ?',
	'Person Responsible':'who was responsible for the second task  of the first agenda?',
	'Deadline':'when is the deadline of the second task of the first agenda?'
  },
 'table_main2': {
    'Discussion Topic':'what was the topic of the second agenda?',
	'Presenter':'who was the presenter of the second agenda?',
	'Conclusion':'what was the conclusion of the second agenda?'
 },
  'table_main2_1':{
	'Action Item':'what was the first task for the second agenda ?',
	'Person Responsible':'who was responsible for the first task  of the second agenda?',
	'Deadline':'when is the deadline of the first task of the second agenda?'
  },
    'table_main2_2':{
	'Action Item':'what was the second task for the second agenda ?',
	'Person Responsible':'who was responsible for the second task  of the second agenda?',
	'Deadline':'when is the deadline of the second task of the second agenda?'
  }
}


## number  처리할것 
#질문 부분 수정
class make_tables:

	def __init__(self, queries_dict, full_context, type='PPC'):
		self.date = full_context[:10] 
		self.full_context =  full_context[22:]
		# if type=='PPC':
		# 	self.num_act_items, self.num_agendas = list(MRC_EXTRACTION.get_topN_answers( \
		# 		['How many action items for the first agenda?','How many agendas?'], self.full_context).values())  #모델 학습 후 query로 몇개인지 찾기

		#질문 수정할 것
		queries=[list(x.values()) for x in  list(queries_dict.values())]
		queries = [item for sublist in queries for item in sublist]

		mrc = MRC_EXTRACTION(queries, self.full_context)
		
		answers_list=mrc.get_topN_answers()

		temp_df = pd.DataFrame(queries_dict)
		self.answers_dict=temp_df.replace(answers_list).to_dict()
		self.answers_dict['Date']=self.date
		print(answers_list)
		try: 
			self.num_agendas=w2n.word_to_num(self.answers_dict['table_top']['num_agendas'])
			self.num_act_items={}
			for i in range(1, 1+self.num_agendas):
				try: 
					self.num_act_items[i]=w2n.word_to_num(self.answers_dict['table_top']['num_act_items_'+str(i)])
				except:
					self.num_act_items[i]=0
		except:
			self.num_agendas=0



	# 회의록 공통 질문 및 agenda 내 첫번째 table용
	def make_table_top(self, items=['Topic','Participants', 'Date'], loc='table_top'):

		html_string = '<table class="table table-bordered">'

		for item in items:
			temp='<tr><td><strong>'+item+'</strong></td>'
			html_string+=temp
			if item=='Date':
				answer_str = '<td>'+self.answers_dict[item]+'</td></tr>' 
			else:
				answer_str = '<td>'+self.answers_dict[loc][item]+'</td></tr>' 

			html_string+=answer_str

		html_string+='</table>'

		return html_string


	# agenda의 두번째 table용
	def make_table_mid(self, n_act_items=0,items=['Action Item','Person Responsible', 'Deadline'], loc='table_main1'):

		if n_act_items==0:
			return ''
		html_string = '<table class="table addel table-bordered" id="addel"><tr>'

		for item in items:
			html_string += '<td><strong>'+item+'</strong></td>'

		html_string+='</tr><tr>'

		for i in range(1,1+n_act_items):
			for item in items:
				answer = '<td>'+self.answers_dict[loc+'_'+str(i)][item]+'</td>' # 답 나오게 수정
				html_string += answer

		html_string+='</tr>'

		html_string+='<input type="button" class="float" value="Add" id="addrow2"></table>'

		return html_string


	# 각 Agenda table 생성용

	def make_table_final(self, type='PPC'):

		html_string = self.make_table_top()
		label = ''
		if type=='PPC':  
			label = 'Agenda'
		for i in range(1, 1+self.num_agendas):
			loc='table_main'+str(i)
			html_string += '<label><strong>'+label+' '+str(i)+'</strong></label>'
			html_string += self.make_table_top(items=['Presenter', 'Discussion Topic', 'Conclusion'], loc=loc)# 이후 적절한 query로 수정
			html_string += self.make_table_mid(self.num_act_items[i], loc=loc) # 수정

		html_string += '<table class="table table-bordered"><tr><td><strong>Additional Notes</strong></td></tr><tr><td></td></tr></table>' # 수정
		return html_string




In [ ]:
import time
full="2021.02.14. 16:03 <br> Erin Hi everyone, We're starting 3rd KIC Project Progress Check meeting. <br> The attendees of today's meeting are Erin, Kim, Steven, Sierra, Koo, and Jeon.  <br> Shall we begin our meeting? <br> So we have two main agendas for today. <br> Why don't we start with Kim and talk about the first agenda? <br>  <br> Kim The first thing we need to discuss about is what API we should use. <br> So I have been searching up APIs and found 5 products, but the ones worth talking about are 2, one made by Google and the other made by IBM. Both of them have pretty much similar quality I think. They can interpret the words well and transcribe them with high accuracy. The bottom line is, Google and IBM works the best.  <br>  <br> Erin Oh I have used Google API before. Isn't it kinda bad? The accuracy is not so great isn't it?  <br>  <br> Kim I think the accuracy and the level of completion is different between the free version and premium paid version. Since we can still use the premium paid version for free during trial period, we should do that. <br>  <br> Erin Alright then, could you please use the premium version and see how it works out? After that let's select the API and do some coding works to get them to function. You should work with Sierra on that Can we do this by next meeting which we should hold on the 12th? <br>  <br> Kim Okay, so we have two action items for this. First thing is to evaluate Google Premium API, and i will do that. The next item is to confirm API and edit the code, me, Kim and Sierra will <br>  <br> Erin Next is Steven, please update us with your interface works, the second agenda for this meeting.  <br>  <br> Steven Well I haven't been very productive with my task so please excuse me. What I did do however is I searched up some already-available services for Ai automated transcription services. There were two that I found to be somewhat relevant to what we were doing: Otter.ai and Deeptalk. Fortunately both of them have yet developed ways to produce meeting notes or minute so if we can get it done, we would be the first to do so. I will get the interface design by today and the rest of the work including homepage feedback by the next meeting on the 12th. <br>  <br> Erin alright. Steven will have two tasks for the second agenda. First one is to get the interface design by today. Second is to give some Homepage feedback by the next meeting. <br> "
start=time.time()
mtl=make_tables(query_dict_ex, full)
result=mtl.make_table_final()
print(time.strftime("%H:%M:%S", time.gmtime(time.time()-start)))
result

{'who are the attendees at the meeting?': 'Erin, Kim, Steven, Sierra, Koo, and Jeon.', 'what was the main topic of the meeting?': 'KIC Project Progress Check', 'How many agendas?': 'two', 'How many action items for the first agenda?': 'two', 'How many action items for the second agenda?': 'two', 'what was the topic of the first agenda?': 'what API we should use.', 'who was the presenter of the first agenda?': 'Kim', 'what was the conclusion of the first agenda?': 'Google and IBM works the best.', 'what was the first task for the first agenda ?': 'evaluate Google Premium API,', 'who was responsible for the first task  of the first agenda?': 'Kim', 'when is the deadline of the first task of the first agenda?': 'today.', 'what was the second task for the first agenda ?': 'to confirm API and edit the code,', 'who was responsible for the second task  of the first agenda?': 'Steven,', 'when is the deadline of the second task of the first agenda?': 'by the next meeting on the 12th.', 'what wa

'<table class="table table-bordered"><tr><td><strong>Topic</strong></td><td>KIC Project Progress Check</td></tr><tr><td><strong>Participants</strong></td><td>Erin, Kim, Steven, Sierra, Koo, and Jeon.</td></tr><tr><td><strong>Date</strong></td><td>2021.02.14</td></tr></table><label><strong>Agenda 1</strong></label><table class="table table-bordered"><tr><td><strong>Presenter</strong></td><td>Kim</td></tr><tr><td><strong>Discussion Topic</strong></td><td>what API we should use.</td></tr><tr><td><strong>Conclusion</strong></td><td>Google and IBM works the best.</td></tr></table><table class="table addel table-bordered" id="addel"><tr><td><strong>Action Item</strong></td><td><strong>Person Responsible</strong></td><td><strong>Deadline</strong></td></tr><tr><td>evaluate Google Premium API,</td><td>Kim</td><td>today.</td><td>to confirm API and edit the code,</td><td>Steven,</td><td>by the next meeting on the 12th.</td></tr><input type="button" class="float" value="Add" id="addrow2"></table><

In [ ]:
with open("Output.txt", "w") as text_file:
    text_file.write(result)